In [2]:
pip install prettytable==3.6.0 --upgrade

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install duckdb

In [1]:
import os
import pandas as pd
import duckdb

# Step 3: Set your dataset folder path
path_dir = r"C:\Users\COMPUTER POINT\Desktop\olist_ecommerce_dataset"

# Step 4: Connect to a DuckDB database file (or use ':memory:' for temp session)
con = duckdb.connect("olist_ecommerce_duckdb_new.db")  # file-based DB (you can open again later)

In [2]:
for file in os.listdir(path_dir):
    if file.endswith(".csv"):
        file_path = os.path.join(path_dir, file)
        df = pd.read_csv(file_path)
        table_name = file.replace(".csv", "")
        con.register(table_name, df)  # creates temp table/view
        con.execute(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM {table_name}")

In [10]:
con.execute("SHOW TABLES").df()

,name
0,olist_customers_dataset
1,olist_customers_dataset
2,olist_geolocation_dataset
3,olist_geolocation_dataset
4,olist_order_items_dataset
5,olist_order_items_dataset
6,olist_order_payments_dataset
7,olist_order_payments_dataset
8,olist_order_reviews_dataset
9,olist_order_reviews_dataset


In [6]:
con.execute("""
    SELECT order_status, COUNT(*) as total_orders
    FROM olist_orders_dataset
    GROUP BY order_status
    ORDER BY total_orders DESC
""").df()

,order_status,total_orders
0,delivered,96478
1,shipped,1107
2,canceled,625
3,unavailable,609
4,invoiced,314
5,processing,301
6,created,5
7,approved,2


In [11]:
#SALES & REVENUE ANALYSIS
#From Tables:
#olist_orders_dataset
#olist_order_payments_dataset
#olist_order_items_dataset

#Analysis Ideas:

#🗓️ Monthly revenue trend

#💰 Average revenue per order

#💳 Payment types used and % of total

#📦 Top product categories by revenue

#💸 Revenue per seller

#📍 Revenue by customer city/state




In [12]:
con.execute("""
    SELECT 
    SUM(CASE WHEN order_status = 'delivered' THEN payment_value ELSE 0 END) AS total_revenue_for_delivered_goods,
    SUM(CASE WHEN order_status != 'delivered' THEN payment_value ELSE 0 END) AS total_revenue_for_undelivered_goods, 
    COUNT(DISTINCT(order_id)) AS total_num_orders
    FROM olist_order_payments_dataset
    INNER JOIN olist_orders_dataset USING(order_id)
    INNER JOIN olist_order_items_dataset USING(order_id)

    """).df()
    

,total_revenue_for_delivered_goods,total_revenue_for_undelivered_goods,total_num_orders
0,1.977616e+07,531974.27,98665


In [13]:
con.execute("""
    SELECT 
        DATE_PART('year', CAST(od.order_purchase_timestamp AS TIMESTAMP)) AS year,
        DATE_PART('month', CAST(od.order_purchase_timestamp AS TIMESTAMP)) AS month,
        SUM( DISTINCT op.payment_value) AS total_revenue
    FROM olist_order_items_dataset AS oi
    LEFT JOIN olist_order_payments_dataset AS op USING(order_id)
    LEFT JOIN olist_orders_dataset AS od USING(order_id)
    WHERE order_status = 'delivered'
    GROUP BY year, month
    ORDER BY year, month ASC 
""").df()

,year,month,total_revenue
0,2016,9,347.52
1,2016,10,73914.58
2,2016,12,19.62
3,2017,1,187779.41
4,2017,2,344134.79
5,2017,3,526961.66
6,2017,4,505665.53
7,2017,5,724504.55
8,2017,6,600753.27
9,2017,7,737293.08


In [14]:
con.execute("""
SELECT SUM(op.payment_value)/COUNT(DISTINCT(order_id)) AS AVG_REV_PER_ORDER 
    
    FROM olist_order_items_dataset AS oi
    LEFT JOIN olist_order_payments_dataset AS op USING(order_id)
    LEFT JOIN olist_orders_dataset AS od USING(order_id)
    
""").df()

,AVG_REV_PER_ORDER
0,205.82708


In [15]:
con.execute("""
SELECT DISTINCT(payment_type), COUNT(*) OVER (PARTITION BY payment_type) as num_payments, 
ROUND(COUNT(*) over (PARTITION BY payment_type)/COUNT(*) OVER(),2)*100 AS percent_payment_values
FROM olist_order_payments_dataset

    
""").df()

,payment_type,num_payments,percent_payment_values
0,credit_card,76795,74.0
1,voucher,5775,6.0
2,not_defined,3,0.0
3,debit_card,1529,1.0
4,boleto,19784,19.0


In [5]:
con.execute("""
SELECT product_category_name_english as category_name , SUM(payment_value) AS rev_per_category

FROM olist_order_items_dataset as oi
LEFT JOIN olist_order_payments_dataset as op USING (order_id)
LEFT JOIN olist_products_dataset as pd USING (product_id)
LEFT JOIN product_category_name_translation as pc USING (product_category_name)
GROUP BY product_category_name_english
ORDER BY rev_per_category DESC
LIMIT 10

""").df()

,category_name,rev_per_category
0,bed_bath_table,1712553.67
1,health_beauty,1657373.12
2,computers_accessories,1585330.45
3,furniture_decor,1430176.39
4,watches_gifts,1429216.68
5,sports_leisure,1392127.56
6,housewares,1094758.13
7,auto,852294.33
8,garden_tools,838280.75
9,cool_stuff,779698.00


In [17]:
con.execute("""
SELECT seller_id, SUM(payment_value) AS REV_PER_SELLER

FROM olist_order_items_dataset as oi 
LEFT JOIN olist_order_payments_dataset as op USING (order_id)
GROUP BY seller_id
ORDER BY REV_PER_SELLER DESC
LIMIT 10
""").df()

,seller_id,REV_PER_SELLER
0,7c67e1448b00f6e969d365cea6b010ab,507166.91
1,1025f0e2d44d7041d6cf58b6550e0bfa,308222.04
2,4a3ca9315b744ce9f8e9374361493884,301245.27
3,1f50f920176fa81dab994f9023523100,290253.42
4,53243585a1d6dc2643021fd1853d8905,284903.08
5,da8622b14eb17ae2831f4ac5b9dab84a,272219.32
6,4869f7a5dfa277a7dca6462dcf3b52b2,264166.12
7,955fee9216a65b617aa5c0531780ce60,236322.30
8,fa1c13f2614d7b5c4749cbc52fecda94,206513.23
9,7e93a43ef30c4f03f38b393420bc753a,185134.21


In [18]:
con.execute("""
SELECT customer_state, customer_city , SUM(payment_value) AS revenue_per_state_city

FROM olist_orders_dataset
LEFT JOIN olist_customers_dataset  USING(customer_id)
LEFT JOIN olist_order_payments_dataset USING (order_id)
GROUP BY customer_state, customer_city
ORDER BY revenue_per_state_city DESC
LIMIT 10
""").df()

,customer_state,customer_city,revenue_per_state_city
0,SP,sao paulo,2203373.09
1,RJ,rio de janeiro,1161927.36
2,MG,belo horizonte,421765.12
3,DF,brasilia,354216.78
4,PR,curitiba,247392.48
5,RS,porto alegre,224731.42
6,BA,salvador,218071.50
7,SP,campinas,216248.43
8,SP,guarulhos,165121.99
9,RJ,niteroi,139996.99


In [ ]:
#Customer Behavior & Segmentation
#Goal: Understand how different customers behave.

# Total customers done

# Orders per customer (RFM style) done

 #% of returning vs. one-time customers no repeat customer

# Avg. spend per customer done

# Customer location (state/city) done

# Churn indicators (last purchase date, inactivity) no data

#Tables:
#olist_customers_dataset, olist_orders_dataset, olist_geolocation_dataset

In [30]:
con.execute("""
SELECT COUNT(DISTINCT customer_id) as TOTAL_UNIQUE_CUSTOMERS
FROM olist_customers_dataset
""").df()

,TOTAL_UNIQUE_CUSTOMERS
0,99441


In [20]:
con.execute("""
SELECT DISTINCT(customer_id), SUM(payment_value) OVER (PARTITION BY customer_id) AS rev_per_customer, 
       COUNT(order_id) OVER(PARTITION BY customer_id) AS num_orders, 
       DATE_DIFF( 'day' , MAX(CAST(order_purchase_timestamp AS DATE)) OVER (PARTITION BY customer_id) , CURRENT_DATE) AS days_since_last_order

FROM olist_customers_dataset
LEFT JOIN olist_orders_dataset USING (customer_id)
LEFT JOIN olist_order_payments_dataset USING (order_id)
ORDER BY  days_since_last_order, num_orders, rev_per_customer DESC
LIMIT 10
""").df()

,customer_id,rev_per_customer,num_orders,days_since_last_order
0,a4b417188addbc05b26b72d5e44837a1,89.71,1,2489
1,856336203359aa6a61bf3826f7d84c49,222.03,1,2490
2,4c2ec60c29d10c34bd49cb88aa85cfc4,197.55,1,2503
3,bf6181a85bbb4115736c0a8db1a53be3,80.38,1,2505
4,2823ffda607a2316375088e0d00005ec,137.03,1,2507
5,55c9dad94ec1a2ba57998bdb376c230a,94.63,1,2510
6,7a8a02e54cf3472df427e927f8b9c046,221.54,1,2511
7,191984a8ba4cbb2145acb4fe35b69664,191.46,1,2516
8,ad2eb5d02c0808bcd8743117f6383cf6,84.58,1,2519
9,6ddc45ad08e6a2c885ca2255fc52d6df,414.04,1,2523


In [21]:
con.execute("""
WITH num_orders AS (
    SELECT customer_id, COUNT(order_id) AS num_orders
    FROM olist_orders_dataset 
    GROUP BY customer_id),
total_orders AS ( 
SELECT 
COUNT(CASE WHEN num_orders = 1 THEN customer_id END) AS cust_with_one,
COUNT(CASE WHEN num_orders > 1 THEN customer_id END) AS cust_with_more, 
COUNT(customer_id) AS total_customers
FROM num_orders )
SELECT total_customers, CAST(cust_with_one AS DOUBLE) / total_customers AS pct_one,
    CAST(cust_with_more AS DOUBLE) / total_customers AS pct_more
FROM total_orders
""").df() #HYPOTHESES that there are only one time customers in olist_datset and repeat customers are 0 

,total_customers,pct_one,pct_more
0,99441,1.0,0.0


In [4]:
con.execute("""
SELECT 
    COUNT(*) AS total_orders,
    COUNT(DISTINCT customer_id) AS total_customers,
    COUNT(*) - COUNT(DISTINCT customer_id) AS extra_orders,
    COUNT(DISTINCT CASE WHEN num_orders > 1 THEN customer_id END) AS repeat_customers
FROM (
    SELECT customer_id, COUNT(order_id) AS num_orders
    FROM olist_orders_dataset
    GROUP BY customer_id
)
""").df()


,total_orders,total_customers,extra_orders,repeat_customers
0,99441,99441,0,0


In [7]:
con.execute("""
SELECT COUNT(*) OVER (PARTITION BY customer_id) AS total_orders,
     customer_id AS unique_customers,
       MIN(order_purchase_timestamp) OVER (PARTITION BY customer_id) AS first_order_date,
       MAX(order_purchase_timestamp) OVER( PARTITION BY customer_id) AS last_order_date
FROM olist_orders_dataset ;

""").df()

,total_orders,unique_customers,first_order_date,last_order_date
0,1,0013cd8e350a7cc76873441e431dd5ee,2018-05-07 23:25:09,2018-05-07 23:25:09
1,1,0017a0b4c1f1bdb9c395fa0ac517109c,2018-03-03 18:44:01,2018-03-03 18:44:01
2,1,002937abdae13680e17dccd3868b4825,2017-11-24 20:26:39,2017-11-24 20:26:39
3,1,0030aded3a67b29946a128ee8468388e,2017-11-23 21:18:14,2017-11-23 21:18:14
4,1,0031abfb953b66e998f67b09e7b11375,2018-01-23 13:57:55,2018-01-23 13:57:55
...,...,...,...,...
99436,1,ffc261b1c1e12a951ee0fc9d27463d3d,2017-02-21 16:17:51,2017-02-21 16:17:51
99437,1,ffcd14ed59af25129b69e7225a599db9,2018-01-06 16:20:01,2018-01-06 16:20:01
99438,1,fff25ab9d3c9ba6ad8119576f285a3d9,2017-05-04 11:08:52,2017-05-04 11:08:52
99439,1,fff39f01a81d96b066b1893ad395a98b,2018-07-01 11:49:01,2018-07-01 11:49:01


In [ ]:
#Order Performance & Fulfillment
#Goal: Track delivery efficiency and order volume.

# Total orders
# Orders per month/year
# Order status breakdown (delivered, canceled, etc.)
# Delivery duration: estimated vs. actual
# % of late deliveries
# Average delivery time

# Tables:
#olist_orders_dataset, olist_order_items_dataset

#Hypothesis:

#High late delivery rates could affect customer satisfaction and repeat business.

In [76]:
con.execute("""
SELECT COUNT(order_id) as num_orders, DATE_PART( 'month' ,  CAST (order_purchase_timestamp AS DATE) ) AS month, 
DATE_PART('year', CAST( order_purchase_timestamp AS DATE)) as year

FROM olist_orders_dataset 
GROUP BY ROLLUP( year, month)
ORDER BY year, month DESC

""").df()


,num_orders,month,year
0,1,12,2016
1,324,10,2016
2,4,9,2016
3,329,<NA>,2016
4,5673,12,2017
5,7544,11,2017
6,4631,10,2017
7,4285,9,2017
8,4331,8,2017
9,4026,7,2017


In [80]:
con.execute("""
SELECT 
COUNT(CASE WHEN order_status = 'delivered' THEN order_id END) AS delivered, 
COUNT(CASE WHEN order_status = 'shipped' THEN order_id END) AS shipped,
COUNT(CASE WHEN order_status = 'processing' THEN order_id END) AS processing,
COUNT(CASE WHEN order_status = 'created' THEN order_id END) AS created,
COUNT(CASE WHEN order_status = 'invoiced' THEN order_id END) AS invoiced,
COUNT(CASE WHEN order_status = 'unavailable' THEN order_id  END) AS unavailable,
COUNT(CASE WHEN order_status = 'canceled' THEN order_id  END) AS canceled,
COUNT(CASE WHEN order_status = 'approved' THEN order_id  END) AS approved,
DATE_PART( 'month' ,  CAST (order_purchase_timestamp AS DATE) ) AS month, 
DATE_PART('year', CAST( order_purchase_timestamp AS DATE)) as year

FROM olist_orders_dataset 
GROUP BY month, year,
ORDER BY year, month DESC 
LIMIT 10
""").df()


,delivered,shipped,processing,created,invoiced,unavailable,canceled,approved,month,year
0,1,0,0,0,0,0,0,0,12,2016
1,265,8,2,0,18,7,24,0,10,2016
2,1,1,0,0,0,0,2,0,9,2016
3,5513,57,35,2,13,42,11,0,12,2017
4,7289,72,25,2,35,84,37,0,11,2017
5,4478,33,20,0,16,58,26,0,10,2017
6,4150,38,22,0,17,38,20,0,9,2017
7,4193,41,18,0,20,32,27,0,8,2017
8,3872,56,11,0,7,52,28,0,7,2017
9,3135,47,12,0,11,24,16,0,6,2017


In [92]:
con.execute("""
WITH delivery AS (SELECT DISTINCT order_id,
DATE_DIFF('day',  CAST(order_purchase_timestamp AS DATE),  CAST(order_estimated_delivery_date AS DATE)) AS estimated_delivery_days,
DATE_DIFF('day',  CAST(order_purchase_timestamp AS DATE),  CAST(order_delivered_customer_date AS DATE)) AS actual_delivery_days,
DATE_DIFF('day',  CAST(order_delivered_customer_date AS DATE), CAST(order_estimated_delivery_date AS DATE)) AS delivery_delay

FROM olist_orders_dataset )

SELECT order_id, estimated_delivery_days, actual_delivery_days, 
CASE WHEN delivery_delay > 0 THEN CONCAT(delivery_delay, ' ', 'days before') 
WHEN delivery_delay = 0 THEN 'on time'
ELSE CONCAT(delivery_delay, ' ', 'days later') END AS RESULTED_TIME
FROM delivery
""").df()

,order_id,estimated_delivery_days,actual_delivery_days,RESULTED_TIME
0,ecab90c9933c58908d3d6add7c6f5ae3,47,30,17 days before
1,989225ba6d0ebd5873335f7e01de2ae7,27,22,5 days before
2,7206b86ea789983f7a273ea7fa0bc2a8,17,10,7 days before
3,9faeb9b2746b9d7526aef5acb08e2aa0,21,5,16 days before
4,a685d016c8a26f71a0bb67821070e398,17,24,-7 days later
...,...,...,...,...
99436,b76830e649f13dfd6d806bb6be2b5406,16,10,6 days before
99437,ea9215d79709bb4917f0d0976eb91d17,28,15,13 days before
99438,a281208da836ef2c9d23718a7a855a92,21,6,15 days before
99439,7f19df33207d507c5b83aef798aff0d2,28,21,7 days before


In [15]:
con.execute("""
WITH delivery AS (SELECT DISTINCT order_id,
DATE_DIFF('day',  CAST(order_purchase_timestamp AS DATE),  CAST(order_estimated_delivery_date AS DATE)) AS estimated_delivery_days,
DATE_DIFF('day',  CAST(order_purchase_timestamp AS DATE),  CAST(order_delivered_customer_date AS DATE)) AS actual_delivery_days,
DATE_DIFF('day',  CAST(order_delivered_customer_date AS DATE), CAST(order_estimated_delivery_date AS DATE)) AS delivery_delay

FROM olist_orders_dataset )

SELECT  COUNT(CASE WHEN actual_delivery_days > estimated_delivery_days THEN CAST(order_id AS VARCHAR) END) * 100.0 / COUNT(order_id) AS perc_late_delivery,
 COUNT(CASE WHEN actual_delivery_days < estimated_delivery_days THEN CAST(order_id AS VARCHAR) END) * 100.0 / COUNT(order_id) AS perc_early_delivery,
 COUNT(CASE WHEN actual_delivery_days = estimated_delivery_days THEN CAST(order_id AS VARCHAR) END) * 100.0 / COUNT(order_id) AS perc_ontime_delivery
FROM delivery
""").df()

,perc_late_delivery,perc_early_delivery,perc_ontime_delivery
0,6.571736,89.147334,1.299263


In [18]:
con.execute("""
WITH delivery AS (SELECT DISTINCT order_id,
DATE_DIFF('day',  CAST(order_purchase_timestamp AS DATE),  CAST(order_estimated_delivery_date AS DATE)) AS estimated_delivery_days,
DATE_DIFF('day',  CAST(order_purchase_timestamp AS DATE),  CAST(order_delivered_customer_date AS DATE)) AS actual_delivery_days,
DATE_DIFF('day',  CAST(order_delivered_customer_date AS DATE), CAST(order_estimated_delivery_date AS DATE)) AS delivery_delay

FROM olist_orders_dataset )

SELECT CONCAT(ROUND(AVG(actual_delivery_days),2),' ', 'days') AS avg_delivery_time
FROM delivery
""").df()

,avg_delivery_time
0,12.5 days


In [ ]:
 #Payment Analysis
#Goal: Analyze customer payment preferences and success.

# Count and % share of each payment_type
# Avg. payment value per type
# Avg. number of installments
# Payment success vs. failed (based on order status)

# Tables:
#olist_order_payments_dataset, joined with olist_orders_dataset

#Hypothesis:

#Some payment types may correlate with higher failure/cancellation rates.

In [24]:
con.execute("""
SELECT 
DISTINCT(payment_type), 
COUNT(*) OVER (PARTITION BY payment_type) as num_payments, 
ROUND(COUNT(*) over (PARTITION BY payment_type)/COUNT(*) OVER(),2)*100 AS percent_payment_values, 
ROUND(AVG(payment_value) OVER (PARTITION BY payment_type),2) as avg_payment_value_type, 
ROUND(AVG(payment_installments) OVER( PARTITION BY payment_type),2) AS AVG_installment_value_type
FROM olist_orders_dataset LEFT JOIN olist_order_payments_dataset USING(order_id)
ORDER BY num_payments DESC
    
""").df()

,payment_type,num_payments,percent_payment_values,avg_payment_value_type,avg_installment_value_type
0,credit_card,76795,74.0,163.32,3.51
1,boleto,19784,19.0,145.03,1.00
2,voucher,5775,6.0,65.70,1.00
3,debit_card,1529,1.0,142.57,1.00
4,not_defined,3,0.0,0.00,1.00
5,None,1,0.0,NaN,NaN


In [43]:
con.execute("""
WITH payment_attempt AS (SELECT 
order_id,
MAX(payment_sequential) AS max_attempt
FROM olist_order_payments_dataset 
GROUP BY order_id
ORDER BY max_attempt DESC)
SELECT CONCAT(ROUND(COUNT(CASE WHEN max_attempt > 1 THEN order_id END)/COUNT(*),4)*100,' ','%') AS retry_payments,
CONCAT(ROUND(COUNT(CASE WHEN max_attempt = 1 THEN order_id END)/COUNT(*),4)*100,' ','%') as successful_payments
FROM payment_attempt
""").df()

,retry_payments,successful_payments
0,3.06 %,96.94 %


In [ ]:
#5. Product & Category Insights
#Goal: Identify top-performing products and sellers.
# Top-selling products by quantity
# Most revenue-generating categories
# Avg. price per category
# Avg. items per order
# Product diversity per seller

In [67]:
con.execute("""
WITH product_category1 AS (SELECT DISTINCT product_category_name_english AS product_category, 
COUNT(*) OVER (PARTITION BY product_category_name_english) AS num_orders_per_category,
SUM(payment_value) OVER (PARTITION BY product_category_name_english) AS rev_per_category, 
AVG(price) OVER (PARTITION BY product_category_name_english) AS avg_price_per_category

FROM olist_order_items_dataset LEFT JOIN olist_products_dataset USING (product_id)
LEFT JOIN product_category_name_translation USING( product_category_name)
LEFT JOIN olist_order_payments_dataset USING (order_id)

ORDER BY num_orderS_per_category DESC
LIMIT 10)
SELECT product_category, num_orders_per_category, DENSE_RANK() OVER (ORDER BY num_orders_per_category DESC) AS RANK_POSITION, rev_per_category,
avg_price_per_category
FROM product_category1 
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,product_category,num_orders_per_category,RANK_POSITION,rev_per_category,price_per_category
0,bed_bath_table,11823,1,1712553.67,92.408950
1,health_beauty,9975,2,1657373.12,130.074263
2,sports_leisure,8945,3,1392127.56,114.476952
3,furniture_decor,8744,4,1430176.39,87.499301
4,computers_accessories,8082,5,1585330.45,116.589652
5,housewares,7355,6,1094758.13,90.630455
6,watches_gifts,6201,7,1429216.68,202.087292
7,telephony,4721,8,486882.05,71.891370
8,garden_tools,4574,9,838280.75,113.296358
9,auto,4379,10,852294.33,140.843231


In [76]:
con.execute("""
SELECT 
    ROUND(AVG(items_per_order), 2) AS avg_items_per_order
FROM (
    SELECT 
        order_id, 
        COUNT(order_item_id) AS items_per_order
    FROM olist_order_items_dataset
    GROUP BY order_id
) sub
""").df()

,avg_items_per_order
0,1.14


In [14]:
con.execute("""
SELECT COUNT( DISTINCT product_category_name_english) AS product_diversity , seller_id

FROM olist_order_items_dataset LEFT JOIN olist_sellers_dataset USING(seller_id)
LEFT JOIN olist_products_dataset USING (product_id) LEFT JOIN product_category_name_translation USING (product_category_name)
GROUP BY seller_id
ORDER BY product_diversity  DESC
LIMIT 10
""").df()

,product_diversity,seller_id
0,27,b2ba3715d723d245138f291a6fe42594
1,23,4e922959ae960d389249c378d1c939f5
2,23,955fee9216a65b617aa5c0531780ce60
3,21,1da3aeb70d7989d1e6d9b0e887f97c23
4,19,f8db351d8c4c4c22c6835c19a46f01b0
5,17,18a349e75d307f4b4cc646a691ed4216
6,15,6edacfd9f9074789dad6d62ba7950b9c
7,15,70a12e78e608ac31179aea7f8422044b
8,14,2ff97219cb8622eaf3cd89b7d9c09824
9,14,fd386aa7bed2af3c7035c65506c9b4a3


In [16]:
con.execute("""
SELECT DISTINCT product_category_name_english , COUNT(seller_id) AS num_seller_per_category

FROM olist_order_items_dataset LEFT JOIN olist_sellers_dataset USING(seller_id)
LEFT JOIN olist_products_dataset USING (product_id) LEFT JOIN product_category_name_translation USING (product_category_name)
GROUP BY product_category_name_english 
ORDER BY num_seller_per_category DESC
LIMIT 10
""").df()

,product_category_name_english,num_seller_per_category
0,bed_bath_table,11115
1,health_beauty,9670
2,sports_leisure,8641
3,furniture_decor,8334
4,computers_accessories,7827
5,housewares,6964
6,watches_gifts,5991
7,telephony,4545
8,garden_tools,4347
9,auto,4235


In [ ]:
# Customer Satisfaction & Reviews
#Goal: Measure satisfaction and feedback trends.

# Average review score
# % of 5-star / 1-star reviews
# Avg. time to review after delivery
# Review score vs. delivery time correlation

# Tables:
#olist_order_reviews_dataset, olist_orders_dataset

In [3]:
con.execute("""
SELECT AVG(review_score) 
FROM olist_order_reviews_dataset
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,avg(review_score)
0,4.086421


In [33]:
con.execute("""
SELECT 
DISTINCT review_score, 
ROUND((COUNT(*) OVER (PARTITION BY review_score)/COUNT(*) OVER())*100,2) AS perc_reviews

FROM olist_order_reviews_dataset
ORDER BY perc_reviews DESC

""").df()


,review_score,perc_reviews
0,5,57.78
1,4,19.29
2,1,11.51
3,3,8.24
4,2,3.18


In [4]:
con.execute("""
SELECT DISTINCT order_id, DATE_DIFF('day', CAST(order_delivered_customer_date AS DATE) , CAST(review_answer_timestamp AS DATE)) AS time_review_post_delivery

FROM olist_orders_dataset LEFT JOIN olist_order_reviews_dataset USING (order_id)
WHERE order_delivered_customer_date IS NOT NULL AND review_answer_timestamp IS NOT NULL AND review_answer_timestamp >= order_delivered_customer_date
ORDER BY time_review_post_delivery DESC

""").df()


,order_id,time_review_post_delivery
0,bb5849f8ba21da43ffa31ea52ba81b37,519
1,a8a2941e92152e0edc39bb862cd6b6e2,513
2,898ded7ecb1e4d92997856bd8b7a8aa1,509
3,3d80d12c0beadfcdcf2b4df407112f3f,472
4,3b39d2ddd9066f382c924cc2c51dc584,447
...,...,...
91400,41ef253cac66335bdb0b51fb4853f04e,0
91401,f353cd55addf6569b4bfa27e1d08367b,0
91402,29d213d4cf25b10351ee72f7456bec18,0
91403,8d877170b83d13fb1a69834a815d0974,0


In [8]:
con.execute("""
WITH review_score AS(SELECT DISTINCT order_id, DATE_DIFF('day', CAST(order_delivered_customer_date AS DATE) , CAST(review_answer_timestamp AS DATE)) AS time_review_post_delivery

FROM olist_orders_dataset LEFT JOIN olist_order_reviews_dataset USING (order_id)
WHERE order_delivered_customer_date IS NOT NULL AND review_answer_timestamp IS NOT NULL AND review_answer_timestamp >= order_delivered_customer_date
ORDER BY time_review_post_delivery DESC)

SELECT CONCAT(ROUND(avg(time_review_post_delivery),2),' ', 'days') as avg_time_review
FROM review_score
""").df()




,avg_time_review
0,3.71 days


In [ ]:
# Seller Performance Goal: Evaluate seller behavior and consistency
# Total sellers 
# Orders per seller 
#Seller location spread 
# Avg. delivery delay per seller
# Revenue per seller
#Tables: olist_sellers_dataset, olist_order_items_dataset, olist_orders_dataset

In [4]:
con.execute("""
SELECT COUNT(DISTINCT seller_id) AS total_sellers
FROM olist_sellers_dataset
""").df()

,total_sellers
0,3095


In [11]:
con.execute("""
SELECT DISTINCT seller_id, COUNT(order_id) OVER (PARTITION BY seller_id) AS num_order_per_seller

FROM olist_orders_dataset LEFT JOIN olist_order_items_dataset USING(order_id)
LEFT JOIN olist_sellers_dataset USING (seller_id)
ORDER BY num_order_per_seller DESC
""").df()


,seller_id,num_order_per_seller
0,6560211a19b47992c3666cc44a7e94c0,2033
1,4a3ca9315b744ce9f8e9374361493884,1987
2,1f50f920176fa81dab994f9023523100,1931
3,cc419e0650a3c5ba77189a1882b7556a,1775
4,da8622b14eb17ae2831f4ac5b9dab84a,1551
...,...,...
3091,d180e07dc1e58c31c3f0cd599dc4d945,1
3092,d4d99b60c4c94cd35acd502cec65dc8d,1
3093,d8b8f2cf9ff6ba0389072541cb42498c,1
3094,f1fdf2d13186575751aa25876536d85c,1


In [12]:
con.execute("""
SELECT DISTINCT (seller_city), COUNT(seller_id) OVER (PARTITION BY seller_city) AS num_sellers


FROM olist_sellers_dataset
WHERE seller_city IS NOT NULL
ORDER BY num_sellers DESC
LIMIT 10
""").df()

,seller_city,num_sellers
0,sao paulo,694
1,curitiba,127
2,rio de janeiro,96
3,belo horizonte,68
4,ribeirao preto,52
5,guarulhos,50
6,ibitinga,49
7,santo andre,45
8,campinas,41
9,maringa,40


In [20]:
con.execute("""
WITH delivery_delay AS (SELECT seller_id, DATE_DIFF('day', CAST(order_estimated_delivery_date AS DATE), CAST(order_delivered_customer_date AS DATE)) as days_delayed

FROM olist_orders_dataset LEFT JOIN olist_order_items_dataset USING (order_id)
WHERE order_delivered_customer_date > order_estimated_delivery_date)

SELECT  DISTINCT (seller_id) , AVG(days_delayed) OVER (PARTITION BY seller_id) AS avg_delay_per_seller
FROM delivery_delay
ORDER BY avg_delay_per_seller DESC
""").df()



,seller_id,avg_delay_per_seller
0,df683dfda87bf71ac3fc63063fba369d,167.0
1,e83c76265fc54bf41eac728805e4da77,159.0
2,4fb41dff7c50136976d1a5cf004a42e2,134.0
3,1782ce2db72c3dda627d5e7c898104e1,132.0
4,5d378b73ab7dd6f0418d743e5dcb0bd1,100.0
...,...,...
1385,9a03a4e9eb0b947652fae4f6121a3f5e,0.0
1386,a20d8058c866dbaeca014d2cd20ac62e,0.0
1387,5a05a16bb50629ee31afab8a6d4c2674,0.0
1388,daf565f087a52def0fd5c5825b3661bc,0.0


In [3]:
con.execute("""
SELECT DISTINCT (seller_id) AS unique_sellers , SUM (price+freight_value) OVER(PARTITION BY seller_id) AS rev_per_seller

FROM olist_order_items_dataset LEFT JOIN olist_order_payments_dataset USING(order_id)
ORDER BY rev_per_order DESC
LIMIT 10
""").df()

BinderException: Binder Error: Referenced column "rev_per_order" not found in FROM clause!
Candidate bindings: "order_item_id", "olist_order_items_dataset.order_id", "olist_order_payments_dataset.order_id", "seller_id", "freight_value"

LINE 5: ORDER BY rev_per_order DESC
                 ^

In [ ]:
# Geo Analysis Goal: Explore spatial patterns. 
#Customer distribution by state/city 
#Seller distribution 
#Delivery time by region 
#Revenue by region


In [39]:
con.execute("""

SELECT customer_city, COUNT(customer_id) as num_customers

FROM olist_customers_dataset
GROUP BY customer_city
ORDER BY num_customers DESC
LIMIT 10
""").df()

,customer_city,num_customers
0,sao paulo,15540
1,rio de janeiro,6882
2,belo horizonte,2773
3,brasilia,2131
4,curitiba,1521
5,campinas,1444
6,porto alegre,1379
7,salvador,1245
8,guarulhos,1189
9,sao bernardo do campo,938


In [9]:
con.execute("""

SELECT  seller_city, seller_state, COUNT( DISTINCT seller_id) as num_sellers

FROM olist_sellers_dataset
GROUP BY seller_city, seller_state
ORDER BY num_sellers DESC
""").df()


,seller_city,seller_state,num_sellers
0,sao paulo,SP,694
1,curitiba,PR,124
2,rio de janeiro,RJ,93
3,belo horizonte,MG,66
4,ribeirao preto,SP,52
...,...,...,...
631,garulhos,SP,1
632,pinhalzinho,SP,1
633,sao paulop,SP,1
634,santa rosa de viterbo,SP,1


In [17]:
con.execute("""

SELECT customer_city, customer_state, AVG(DATE_DIFF('day', CAST(order_purchase_timestamp AS DATE), CAST(order_delivered_customer_date AS DATE))) as delivery_days

FROM olist_orders_dataset LEFT JOIN olist_customers_dataset USING (customer_id) 
GROUP BY customer_city, customer_state
ORDER BY delivery_days DESC
LIMIT 10
""").df()

,customer_city,customer_state,delivery_days
0,novo brasil,ES,148.0
1,capinzal do norte,MA,109.0
2,adhemar de barros,PR,98.0
3,arace,ES,86.5
4,santa maria,RJ,84.0
5,marcelino vieira,RN,78.0
6,serra dos dourados,PR,75.0
7,aramari,BA,74.0
8,humildes,BA,72.0
9,santa cruz de goias,GO,67.5


In [13]:
con.execute("""
SELECT SUM(payment_value) AS rev_per_state, customer_state

FROM olist_order_payments_dataset as op 
LEFT JOIN olist_orders_dataset as od USING (order_id)
LEFT JOIN olist_customers_dataset USING (customer_id)
GROUP BY customer_state
ORDER BY rev_per_state DESC
LIMIT 10
""").df()




,rev_per_state,customer_state
0,5998226.96,SP
1,2144379.69,RJ
2,1872257.26,MG
3,890898.54,RS
4,811156.38,PR
5,623086.43,SC
6,616645.82,BA
7,355141.08,DF
8,350092.31,GO
9,325967.55,ES
